In [2]:
import pandas as pd

/Users/giorgia/.pyenv/versions/3.7.8/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
def get_daily_cases(data, col_name='series_1'):
    series = data.copy()
    series = series.groupby(by='Date_statistics').count()
    series = series.rename(columns={'Date_file': col_name})
    return series[col_name]

In [4]:
#Load Data
measures = pd.read_excel('measures_strictness.xlsx', engine='openpyxl').rename(columns={'Unnamed: 0':'Date'})
cases_national= pd.read_csv('Data/COVID-19_casus_landelijk.csv', sep=';')
behaviour= pd.read_csv('Data/COVID-19_gedrag.csv', sep=';')

In [6]:
# Create datasets

# Put date in the right format, merge with measures data
cases_national['Date_statistics'] = pd.to_datetime(cases_national['Date_statistics'],errors='ignore', dayfirst=True)
daily_cases = get_daily_cases(cases_national, 'Number of new cases').to_frame().reset_index()
daily_cases['Date_statistics'] = pd.to_datetime(daily_cases['Date_statistics'])
daily_data = daily_cases.merge(measures, left_on='Date_statistics', right_on='Date', how='left').drop(['Date', 'stringency_index', 'government_response_index', 'containment_health_index', 'economic_support_index'], axis=1)

# Put date in the right format, filter to simplify the dataset, drop useless columns
behaviour['Date_of_measurement'] = pd.to_datetime(behaviour['Date_of_measurement'],errors='ignore')
behaviour = behaviour.loc[lambda d: (d.Region_name=='Nederland') & (d.Subgroup == 'Totaal') & (d.Sample_size != 0) & ((d.Indicator_category.str.contains('Naleving')) | (d.Indicator_category.str.contains('Zorgen')))]
behaviour= behaviour.drop(['Date_of_report', 'Wave','Region_code', 'Subgroup', 'Figure_type', 'Region_name', 'Subgroup_category', 'Lower_limit', 'Upper_limit', 'Change_wrt_previous_measurement'], axis=1)

In [8]:
daily_data.to_csv('daily_data.csv', index=False)

In [221]:
behaviour.head()

,Date_of_measurement,Indicator_category,Indicator,Sample_size,Value
1656,2020-09-28,Naleving,Bij_klachten_blijf_thuis,929,46.1
1692,2020-09-28,Naleving,Bij_klachten_laat_testen,1012,32.2
1836,2020-09-28,Naleving,Houd_1_5m_afstand,4966,61.6
1872,2020-09-28,Naleving,Ontvang_max_bezoekers_thuis,5025,97.3
1944,2020-09-28,Naleving,Vermijd_drukke_plekken,5025,63.3


In [8]:
behaviour.groupby('Date_of_measurement').agg(count_of_measurements=pd.NamedAgg(column='Indicator_category', aggfunc='count'))

,count_of_measurements
Date_of_measurement,
2020-09-28,7
2020-10-19,11
2020-11-09,11
2020-11-30,12
2020-12-14,12
2021-01-04,12
2021-01-25,13
2021-02-15,13
2021-03-08,13
